In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Dropout, Activation
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
TB=0
TE=3389
INPUT_SIZE=7
OUTPUT_SIZE=1
BATCH_SIZE=1
TIME_STEPS=1
CELL_SIZE=7
HIDD=32

In [3]:
f=open('test.csv') 
df=pd.read_csv(f)     
data=df.iloc[:,2:10].values

In [4]:
def get_train_data(train_begin=TB,train_end=TE):
    global train_mean, train_std
    data_train=data[train_begin:train_end]
    train_mean=np.mean(data_train,axis=0)
    train_std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-train_mean)/train_std
    train_x = normalized_train_data[:,:7]
    train_y = normalized_train_data[:,7,np.newaxis]
    return train_x[:, np.newaxis], train_y[:, np.newaxis]
train_x, train_y=get_train_data()

In [5]:
def get_test_data(test_begin=TE):
    global test_mean, test_std
    data_test=data[test_begin:]
    test_mean=np.mean(data_test,axis=0)
    test_std=np.std(data_test,axis=0)
    test_data=(data_test-test_mean)/test_std
    test_x=test_data[:,:7]
    test_y=test_data[:,7,np.newaxis]
    return test_x[:, np.newaxis], test_y[:, np.newaxis]
test_x, test_y=get_test_data()

In [6]:
model = Sequential()

model.add(Dense(batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    output_dim=CELL_SIZE,
))

model.add(Dropout(0.2))

model.add(LSTM(HIDD,
    return_sequences=True,      # True: output at all steps. False: output as last step.
    stateful=True
))

model.add(Dropout(0.2))

model.add(Dense(units=1, activation='relu'))

model.compile(optimizer='adam',loss='mse')

train_history=model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=501, verbose=2)

C:\Users\Administrator\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, batch_input_shape=(1, 1, 7))`
  after removing the cwd from sys.path.


Epoch 1/501
12s - loss: 0.5660
Epoch 2/501


KeyboardInterrupt: 

In [ ]:
trainScore = model.evaluate(train_x, train_y, batch_size=BATCH_SIZE, verbose=1)
print(trainScore)

In [ ]:
predict=model.predict(test_x, batch_size=BATCH_SIZE,verbose=1)
print(predict)

In [ ]:
predict.reshape(-1)
np.array(predict)*test_std[3]+test_mean[3]